[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

# Aula 3 — Datas, Reshapes, Joins e Curva Epidêmica

## Objetivos de aprendizagem
- Parsear datas e usar `.dt` (dia/semana/mês/ano; ISO week).
- Agregar por dia/semana com `groupby`/`resample` e criar janelas móveis.
- Reestruturar dados com `melt`, `pivot` e `pivot_table` (wide ↔ long).
- Realizar `joins` para incorporar **população por região** e calcular **incidência**.
- Construir e interpretar uma **curva epidêmica** diária/semana epidemiológica.


In [ ]:
#@title Instalar/Carregar dependências
# Usaremos bibliotecas já disponíveis no Colab.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)

# Semente para reprodutibilidade
rng = np.random.default_rng(42)
print("Ambiente carregado.")

## 1) Dados sintéticos e dicionário de variáveis

Vamos gerar um **line list** sintético: `id`, `age`, `sex`, `region`, `onset_date`, `report_date`, `hospitalized`.  
Criaremos também uma `pop` por **região** para taxas/100 mil.

**Dicionário:**
- `id` (int) — identificador do caso
- `age` (int) — idade em anos
- `sex` ('F'/'M')
- `region` ('Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste')
- `onset_date` (datetime) — data de início dos sintomas
- `report_date` (datetime) — data de notificação
- `hospitalized` ('internado'/'nao_internado')


In [ ]:
# Funções utilitárias
from datetime import datetime

def generate_synthetic_line_list(n=1200, seed=42):
    rng = np.random.default_rng(seed)
    ids = np.arange(1, n+1)
    ages = rng.integers(0, 90, size=n)
    sex = rng.choice(['F', 'M'], size=n, p=[0.53, 0.47])
    regions = rng.choice(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'], size=n, p=[0.08,0.28,0.43,0.14,0.07])
    start = np.datetime64('2023-01-01')
    onset = start + rng.integers(0, 300, size=n).astype('timedelta64[D]')
    report_delay = rng.integers(0, 10, size=n).astype('timedelta64[D]')
    report = onset + report_delay
    hosp = rng.choice(['internado', 'nao_internado'], size=n, p=[0.18, 0.82])
    return pd.DataFrame({
        'id': ids,
        'age': ages,
        'sex': sex,
        'region': regions,
        'onset_date': pd.to_datetime(onset),
        'report_date': pd.to_datetime(report),
        'hospitalized': hosp
    })

def try_load_opendatasus_srag(sample_n=20000):
    """Tenta ler amostra do SIVEP‑Gripe (OpenDataSUS).
    Usa requests com cabeçalho User-Agent para reduzir 403.
    Retorna DataFrame normalizado OU None em falha.
    """
    url = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2023/INFLUD23-26-06-2025.csv'
    try:
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=60)
        resp.raise_for_status()
        # print(resp.content)
        df_raw = pd.read_csv(io.BytesIO(resp.content), sep=';', encoding='latin1', nrows=sample_n, low_memory=False)

        if df_raw is None or df_raw.empty:
            print('Falha ao carregar OpenDataSUS: df_raw is None or empty')
            return None

        colmap = {}

        for c in ['CS_SEXO','SEXO']:
            if c in df_raw.columns: colmap[c] = 'sex'
        for c in ['NU_IDADE_N','NU_IDADE']:
            if c in df_raw.columns: colmap[c] = 'age'
        for c in ['DT_SIN_PRI','DT_SINTOMA']:
            if c in df_raw.columns: colmap[c] = 'onset_date'
        for c in ['DT_NOTIFIC','DT_NOTIF']:
            if c in df_raw.columns: colmap[c] = 'report_date'
        for c in ['SG_UF_NOTA','SG_UF']:
            if c in df_raw.columns: colmap[c] = 'uf'

        df = df_raw.rename(columns=colmap).copy()
        print('Colunas mapeadas:', list(colmap.values()))

        if 'uf' in df.columns:
            uf_to_region = {
                'AC':'Norte','AP':'Norte','AM':'Norte','PA':'Norte','RO':'Norte','RR':'Norte','TO':'Norte',
                'AL':'Nordeste','BA':'Nordeste','CE':'Nordeste','MA':'Nordeste','PB':'Nordeste','PE':'Nordeste','PI':'Nordeste','RN':'Nordeste','SE':'Nordeste',
                'ES':'Sudeste','MG':'Sudeste','RJ':'Sudeste','SP':'Sudeste',
                'PR':'Sul','RS':'Sul','SC':'Sul',
                'DF':'Centro-Oeste','GO':'Centro-Oeste','MT':'Centro-Oeste','MS':'Centro-Oeste'
            }
            df['region'] = df['uf'].map(uf_to_region)

        if 'age' in df.columns:
            df['age'] = pd.to_numeric(df['age'], errors='coerce')
            print('Idade convertida para numérico.')
        if 'sex' in df.columns:
            df['sex'] = df['sex'].astype(str).str.upper().str[0].replace({'F':'F','M':'M'})
            print('Sexo convertido para str e mapeado para F/M.')
        for dcol in ['onset_date','report_date']:
            if dcol in df.columns:
                df[dcol] = pd.to_datetime(df[dcol], errors='coerce', dayfirst=True)
                print(f'Data convertida para datetime: {dcol}')

        hosp_cols = [c for c in df_raw.columns if ('HOSP' in c.upper()) or ('UTI' in c.upper())]
        if hosp_cols:
            df['hospitalized'] = np.where(df_raw[hosp_cols].notna().any(axis=1), 'internado', 'nao_internado')
        else:
            df['hospitalized'] = 'nao_internado'

        keep = [c for c in ['age','sex','region','onset_date','report_date','hospitalized'] if c in df.columns]
        if not keep:
            return None
        return df[keep].dropna(how='all')
    except Exception as e:
        print('Falha ao carregar OpenDataSUS:', e)
        return None

# Carregamento principal com fallback
df_remote = try_load_opendatasus_srag(sample_n=20000)
if df_remote is None or df_remote.empty:
    print('Usando dados SINTÉTICOS (fallback).')
    df = generate_synthetic_line_list(n=1200, seed=42)
else:
    print('Dados do OpenDataSUS (amostra) carregados.')
    df = df_remote.copy()

display(df.head())

# População por região (valores fictícios, mas proporcionais à realidade)
pop = pd.DataFrame({
    'region': ['Norte','Nordeste','Sudeste','Sul','Centro-Oeste'],
    'pop':    [18800000, 57900000, 88000000, 30000000, 16000000]  # aproximado
})

display(pop.head())

## 2) Datas: `to_datetime`, `.dt` e semana ISO

- `pd.to_datetime(...)` para parsing;
- `.dt` para extrair partes: `year`, `month`, `day`, `isocalendar().week`;
- Slices por período com `.between()` e `loc`.


In [ ]:
# Criar colunas auxiliares de datas a partir de onset_date
df = df.assign(
    year = df['onset_date'].dt.year,
    month = df['onset_date'].dt.month,
    iso = df['onset_date'].dt.isocalendar().week.rename('iso_week')
)

# Recorte: Q2 2023 (abril–junho) por onset
mask_q2 = df['onset_date'].between('2023-04-01', '2023-06-30')
df_q2 = df.loc[mask_q2].copy()
print('Linhas no Q2/2023:', df_q2.shape[0])
df_q2[['onset_date','year','month','iso']].head()

**Exercício de fixação (Seção 2)**  
Crie as colunas `report_year`, `report_month` e `report_iso` (a partir de `report_date`) e conte **casos semanais** por `report_iso` (qualquer ano). Mostre as 5 primeiras linhas ordenadas por `report_iso`.


In [ ]:
# Sua resposta aqui


<details><summary><strong>Solução sugerida</strong></summary>

```python
df_fix2 = df.assign(
    report_year = df['report_date'].dt.year,
    report_month = df['report_date'].dt.month,
    report_iso = df['report_date'].dt.isocalendar().week.rename('report_iso')
)
out_fix2 = (df_fix2
            .groupby('report_iso')
            .size()
            .rename('n')
            .reset_index()
            .sort_values('report_iso'))
out_fix2.head()
```

</details>


## 3) Agregações diárias/semanais e média móvel (curva epidêmica básica)

- Contagem diária por `onset_date` (`groupby`/`size`).
- Reamostragem semanal e média móvel de 7 dias.


In [ ]:
# Série diária por onset_date
daily = (df.groupby('onset_date')
           .size()
           .rename('cases')
           .to_frame()
           .sort_index())

# Média móvel de 7 dias
daily['ma7'] = daily['cases'].rolling(7, min_periods=1).mean()

# Série semanal (segunda-feira como anchor por padrão)
weekly = (daily['cases']
          .resample('W-MON')
          .sum()
          .rename('cases_week'))

display(daily.head(), weekly.head())

# Outras opções de séries: 'W-TUE', 'D', '2D', 'W', 'M'. 'A'

# Plot rápido
ax = daily['cases'].plot(figsize=(8,3), label='Diário')
daily['ma7'].plot(ax=ax, label='MM7', linewidth=2)
plt.title('Curva epidêmica — casos por data de início (sintético)')
plt.xlabel('Data'); plt.ylabel('Casos')
plt.legend(); plt.tight_layout()

**Exercício de fixação (Seção 3)**  
Crie uma série **semanal por região**: `onset_week` (`W-MON`) × `region`. Mostre a **tabela longa** com `week`, `region`, `n` (primeiras 10 linhas).


In [ ]:
# Sua resposta aqui
ser = (df.groupby(['region','onset_date']).size()
         .rename('n').to_frame())
display(ser.head())
wk = (ser['n'].resample('W-MON', level='onset_date').sum()
        .rename('n').to_frame()
        .reset_index())
wk.head(10)

<details><summary><strong>Solução sugerida</strong></summary>

```python
ser = (df.groupby(['region','onset_date']).size()
         .rename('n').to_frame())
wk = (ser['n'].resample('W-MON', level='onset_date').sum()
        .rename('n').to_frame()
        .reset_index())
wk.head(10)
```

</details>


## 4) Reshapes: `melt`, `pivot` e `pivot_table`

Vamos transformar uma tabela **semanal por região** (longa) para **wide** e de volta:
- `pivot(index='week', columns='region', values='n')`
- `melt(id_vars=['week'], var_name='region', value_name='n')`


In [ ]:
# Tabela longa semanal por região
week_longa = (df
         .groupby([pd.Grouper(key='onset_date', freq='W-MON', label='left'), 'region'])
         .size()
         .rename('n')
         .reset_index()
         .rename(columns={'onset_date': 'week'}))

# Wide: week × region
wide = (week_longa
        .pivot(index='week', columns='region', values='n')
        .fillna(0)
        .astype(int)
        .sort_index())

# Volta para long
long_again = (wide
              .reset_index()
              .melt(id_vars='week', var_name='region', value_name='n')
              .sort_values(['week', 'region']))

week_longa.head(), wide.head(), long_again.head()

**Exercício de fixação (Seção 4)**  
Usando `week_longa`, crie uma **tabela com totais semanais** (soma entre regiões) via `pivot_table` (ou `groupby`) e mostre as 5 primeiras linhas.


In [ ]:
# Sua resposta aqui
# Dica: pivot table possui uma função agregadora embutida: .pivot_table( ... aggfunc= ...)


<details><summary><strong>Solução sugerida</strong></summary>

```python
tot_sem = (week_longa
           .pivot_table(index='week', values='n', aggfunc='sum')
           .rename(columns={'n':'n_total'}))
tot_sem.head()
```

</details>


## 5) Joins (`merge`) e cálculo de **incidência** por 100 mil

Vamos **juntar** (`merge`) a população por `region` e calcular a **incidência/100 mil**:
- `inc = (n / pop) * 1e5`
- comparar regiões em uma semana.


In [ ]:
# Seleciona uma semana para exemplo (a última disponível)
last_week = week_longa['week'].max()

wk_last = week_longa[week_longa['week'] == last_week].copy()
wk_last = wk_last.merge(pop, on='region', how='left')
wk_last['inc_100k'] = (wk_last['n'] / wk_last['pop']) * 1e5

wk_last.sort_values('inc_100k', ascending=False)

**Exercício de fixação (Seção 5)**  
Calcule a **incidência semanal por 100 mil** para **todas as semanas e regiões** e retorne a **tabela longa** `week, region, n, inc_100k`. Mostre 8 primeiras linhas.


In [ ]:
# Sua resposta aqui


<details><summary><strong>Solução sugerida</strong></summary>

```python
week_longa = wk_lr.merge(pop, on='region', how='left')
wk_all['inc_100k'] = (wk_all['n'] / wk_all['pop']) * 1e5
wk_all.sort_values(['week','region']).head(8)
```

</details>


## 6) Curva epidêmica por região (loop de plots)

Vamos visualizar a curva semanal por região. No Colab, os gráficos aparecem inline.


In [ ]:
regions = week_longa['region'].unique()
fig, axes = plt.subplots(len(regions), 1, figsize=(8, 2.2*len(regions)), sharex=True)
if len(regions) == 1:
    axes = [axes]
for ax, reg in zip(axes, regions):
    s = week_longa[week_longa['region']==reg].set_index('week')['n']
    s.plot(ax=ax)
    ax.set_title(f'Região: {reg} — casos semanais (onset)')
    ax.set_ylabel('Casos')
plt.xlabel('Semana'); plt.tight_layout()

**Exercício de fixação (Seção 6)**  
Crie a coluna `mm3` (média móvel de 3 semanas) em `week_longa` por região e plote `n` × `mm3` para **uma** região à sua escolha.


In [ ]:
# Sua resposta aqui
# Dica: use a função .rolling(...) em um dataframe


<details><summary><strong>Solução sugerida</strong></summary>

```python
reg = 'Sudeste'  # altere se quiser
tmp = week_longa[week_longa['region']==reg].copy().sort_values('week')
tmp['mm3'] = tmp['n'].rolling(3, min_periods=1).mean()

ax = tmp.set_index('week')['n'].plot(figsize=(7,3), label='n')
tmp.set_index('week')['mm3'].plot(ax=ax, label='MM3', linewidth=2)
plt.title(f'{reg}: casos semanais e MM3')
plt.legend(); plt.tight_layout()
```

</details>


## 7) Exercícios finais (7) — dificuldade progressiva

**E1. Seleção + filtro + contagem.**  
Selecione `age`, `sex`, `region` e **conte**, por `region`, o número de casos com `sex == "F"` e `age >= 50` (revisão Aula 2).

**E2. Week x Region wide.**  
A partir de `wk_lr`, gere uma tabela **wide** `week × region` com zeros nos ausentes (revisão reshapes).

**E3. Join + incidência.**  
Una `wk_lr` à população (`pop`) e inclua `inc_100k`. Liste a **semana de maior incidência** para cada região.

**E4. Atraso de notificação.**  
Crie `delay_days = (report_date - onset_date).dt.days`. Para cada região e **semana de onset**, calcule a **proporção** de `delay_days > 5`.

**E5. Faixa etária + hospitalização.**  
Crie `age_group` (0‑4, 5‑11, …, 80+) e calcule, por `age_group`, a **proporção** de `hospitalized == 'internado'`. Mostre o **top‑3**.

**E6. Semana com pico (por região).**  
Para cada região, encontre a semana (em `wk_lr`) com **pico de casos** (empate: escolha a mais recente).

**E7 — Nível Olímpico. Picos sincronizados.**  
1) Construa a tabela wide `week × region` (casos semanais).  
2) Para cada semana, marque `1` se a região está **em pico local** (maior que vizinhas imediatas: semana anterior e posterior); caso borda, considere janela disponível.  
3) Encontre a **semana** com **maior número de regiões** em pico simultaneamente e liste as **regiões** que compõem esse pico sincronizado.


In [ ]:
# E1 — sua resposta


<details><summary><strong>Solução sugerida</strong></summary>

```python
e1 = (df[['age','sex','region']]
        .query('sex == "F" and age >= 50')
        .groupby('region')
        .size()
        .rename('n')
        .reset_index()
        .sort_values('n', ascending=False))
e1
```

</details>


In [ ]:
# E2 — sua resposta


<details><summary><strong>Solução sugerida</strong></summary>

```python
wide_e2 = (week_longa
           .pivot(index='week', columns='region', values='n')
           .fillna(0).astype(int))
wide_e2.head()
```

</details>


In [ ]:
# E3 — sua resposta


<details><summary><strong>Solução sugerida</strong></summary>

```python
e3 = (week_longa.merge(pop, on='region', how='left')
           .assign(inc_100k = lambda d: (d['n']/d['pop'])*1e5))
e3_top = (e3.sort_values(['region','inc_100k'])
            .groupby('region').tail(1)
            .sort_values('inc_100k', ascending=False))
e3_top
```

</details>


In [ ]:
# E4 — sua resposta


<details><summary><strong>Solução sugerida</strong></summary>

```python
df_delay = df.assign(delay_days = (df['report_date'] - df['onset_date']).dt.days)
e4 = (df_delay
      .assign(delay_gt5 = df_delay['delay_days'] > 5)
      .groupby([pd.Grouper(key='onset_date', freq='W-MON'), 'region'])
      .agg(prop_delay_gt5=('delay_gt5','mean'), n=('delay_gt5','size'))
      .reset_index()
      .rename(columns={'onset_date':'week'}))
e4.head(10)
```

</details>


In [ ]:
# E5 — sua resposta


<details><summary><strong>Solução sugerida</strong></summary>

```python
bins = [-1, 4, 11, 17, 29, 39, 49, 59, 69, 79, 200]
labels = ['0-4','5-11','12-17','18-29','30-39','40-49','50-59','60-69','70-79','80+']
e5 = (df.assign(age_group = pd.cut(df['age'], bins=bins, labels=labels))
        .assign(is_hosp = lambda d: d['hospitalized'].eq('internado'))
        .groupby('age_group')
        .agg(prop_hosp=('is_hosp','mean'), n=('is_hosp','size'))
        .reset_index()
        .sort_values('prop_hosp', ascending=False)
        .head(3))
e5
```

</details>


In [ ]:
# E6 — sua resposta


<details><summary><strong>Solução sugerida</strong></summary>

```python
e6 = (week_longa.sort_values(['region','week'])
         .groupby('region')
         .apply(lambda g: g.loc[g['n'].idxmax()])
         .reset_index(drop=True)
         .sort_values('n', ascending=False))
e6
```

</details>


In [ ]:
# E7 — Nível Olímpico: Picos sincronizados em janelas locais
# 1) wide week × region

# 2) pico local: maior que vizinhas imediatas


# 3) semana com mais regiões em pico simultâneo


<details><summary><strong>Solução sugerida</strong></summary>

```python
wide_e7 = (week_longa.pivot(index='week', columns='region', values='n').fillna(0).astype(int))
def local_peaks(series):
    s = series.fillna(0).astype(float).values
    peaks = np.zeros_like(s, dtype=bool)
    for i in range(len(s)):
        left = s[i-1] if i-1 >= 0 else -np.inf
        right = s[i+1] if i+1 < len(s) else -np.inf
        if s[i] > left and s[i] > right:
            peaks[i] = True
    return pd.Series(peaks, index=series.index)
peak_flags = wide_e7.apply(local_peaks)
peak_count = peak_flags.sum(axis=1)
best_week = peak_count.idxmax()
regions_in_peak = peak_flags.columns[peak_flags.loc[best_week]].tolist()
out_e7 = pd.DataFrame({'best_week':[best_week],
                       'num_regions':[int(peak_count.loc[best_week])],
                       'regions':[regions_in_peak]})
out_e7
```

</details>


## 8) Checklist de competências
- [ ] Parsear datas (`to_datetime`) e usar `.dt`/ISO week.
- [ ] Agregar por dia/semana e calcular médias móveis.
- [ ] Reestruturar dados com `melt`/`pivot`/`pivot_table`.
- [ ] Unificação `merge` com população e calculei **incidência/100 mil**.
- [ ] Construção e interpretação com **curvas epidêmicas** (por região).

## 9) Aula 4: Visualizações

## 10) Referências
- `to_datetime`: https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html  
- `dt` accessor: https://pandas.pydata.org/docs/reference/series.html#datetimelike-properties  
- `pivot`/`melt`: https://pandas.pydata.org/docs/user_guide/reshaping.html  
- `merge`: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html  
- `resample`: https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
